In [66]:
import pandas as pd
pd.options.display.max_rows = 150
pd.options.display.max_columns = 150

In [67]:
listing = pd.read_csv('./data/Netherlands/Amsterdam/listings-clean.csv')
reviews = pd.read_csv('./data/Netherlands/Amsterdam/reviews.csv')

In [68]:
listing.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [69]:
listing.shape

(6893, 75)

In [70]:
# QUESTION N°1

quartiers = listing.groupby(by="neighbourhood_cleansed")

# Nombre de review
nb_review = quartiers["number_of_reviews"].sum()

# Nombre de host
nb_host = quartiers['id'].count()
question1 = pd.DataFrame({'nb_host': nb_host, 'nb_review': nb_review})
question1

,nb_host,nb_review
neighbourhood_cleansed,,
Bijlmer-Centrum,94,4964
Bijlmer-Oost,34,983
Bos en Lommer,308,10586
Buitenveldert - Zuidas,143,5691
Centrum-Oost,635,45227
Centrum-West,888,69068
De Aker - Nieuw Sloten,90,6330
De Baarsjes - Oud-West,1001,40372
De Pijp - Rivierenbuurt,710,30418


In [71]:
# QUESTION N°2

# Taux de reponse moyen
moyenne_response = listing["host_response_rate"].str.rstrip('%').astype(float).mean()
moyenne_response

# Taux acceptation moyen
moyenne_accept = listing["host_acceptance_rate"].str.rstrip('%').astype(float).mean()
moyenne_accept

question2 = pd.DataFrame({'mean_response': [moyenne_response], 'mean_accept': [moyenne_accept]}, index=["% par host"])
question2

,mean_response,mean_accept
% par host,94.967341,80.170121


In [72]:
# QUESTION N°3

# Pourcentage type vérification (phone, work_email, email)
phone_verification = listing["host_verifications"].apply(lambda x: 1 if "phone" in x else 0).sum() / listing["host_verifications"].count() * 100
work_email_verification = listing["host_verifications"].apply(lambda x: 1 if "work_email" in x else 0).sum() / listing["host_verifications"].count() * 100
email_verification = listing["host_verifications"].apply(lambda x: 1 if "email" in x and "work_email" != x else 0).sum() / listing["host_verifications"].count() * 100

question3 = pd.DataFrame({'phone_verification': [phone_verification], 'work_email_verification': [email_verification], 'email_verification' : [email_verification]}, index=["% par host"])
question3

,phone_verification,work_email_verification,email_verification
% par host,99.680836,93.108951,93.108951


In [73]:
# QUESTION N°4

# Moyenne d'amenities & écart type
amenities = listing['amenities'].apply(lambda x: str(x)[1:-1].replace('"', '').replace("\\u2013", "-")).str.split(pat=",")
listing['amenities'] = amenities

listing["nb_amenities"] = listing["amenities"].apply(lambda x: len(x))

g_room_type = listing[["room_type","nb_amenities"]].groupby(['room_type']).agg(['mean', 'std'])
g_room_type

nb_amenities           
                        mean        std
room_type                              
Entire home/apt    30.321617  14.252961
Hotel room         22.653333   9.515980
Private room       25.866057  12.292752
Shared room        13.675676   8.100254

In [74]:
# QUESTION N°5
listing["price"] = listing["price"].apply(lambda x: str(x).replace(",", "").replace("$", "")).astype(float)

prix = listing[["room_type","price"]].groupby(["room_type"]).describe()
prix = prix['price'].drop(['count', 'std', 'mean'], axis=1)
prix

,min,25%,50%,75%,max
room_type,,,,,
Entire home/apt,15.0,143.0,200.0,285.0,7900.0
Hotel room,0.0,114.5,222.0,271.0,999.0
Private room,25.0,94.0,130.0,198.0,1657.0
Shared room,45.0,66.0,75.0,94.0,500.0


In [75]:
# QUESTION N°6

listing["bathrooms_text"].value_counts()

1 bath               2795
1.5 baths            1924
1 private bath        847
2 baths               411
1 shared bath         347
1.5 shared baths      198
2.5 baths             157
3 baths                79
3.5 baths              25
0 shared baths         18
Shared half-bath       17
0 baths                15
Half-bath               8
2 shared baths          8
4.5 baths               7
Private half-bath       4
5 baths                 4
4 baths                 4
3 shared baths          4
5.5 baths               2
4 shared baths          2
17 baths                1
3.5 shared baths        1
Name: bathrooms_text, dtype: int64

In [76]:
# On transform nos cases en string
bathrooms = listing["bathrooms_text"]
nb_bathrooms = listing["bathrooms"]

# Ajoute "1" aux 3 cases qui n'ont pas de chiffres
bathrooms = bathrooms.apply(lambda x: "1 {}".format(x) if x == "Shared half-bath" or x == "Half-bath" or x == "Private half-bath" else x).apply(lambda x: str(x)+"s" if str(x)[len(str(x))-1].lower() != "s" else str(x)).replace("nans", 0)

# On sépare le chiffre du Type de bathroom
bathrooms = bathrooms.str.split(' ', 1, expand=True)

# On renome les colonnes
bathrooms = bathrooms.rename(columns={0: "Nombre", 1: "Type"})

# On tranform les str en float de la colonne "Nombre"
bathrooms["Nombre"] = bathrooms["Nombre"].astype(float)

def convertiseur(df):
    """
    Fonction qui multiplie les chiffres de la première colonne en fonction de la deuxième colonne
    : param df : DataFrame
    : return : DataFrame modifié
    """
    result = 0.0
    df[1] = str(df[1]).lower()
    if df[1] == "shared baths" or df[1] == "bath shareds" or df[1] == "half-baths":
        result=df[0]*0.5
    elif df[1] == "private baths":
        result=df[0]*2
    elif df[1] == "half-shareds" or df[1] == "shared half-baths":
        result=df[0]*0.25
    else :
        result = df[0]*1
    return result

bathrooms["Nombre"] = bathrooms.apply(convertiseur,axis=1)
bathrooms = bathrooms.groupby("Type").value_counts().reset_index().rename(columns={0:"count"}).drop("Type", axis=1).groupby("Nombre").sum()
bathrooms = bathrooms.groupby("Nombre").sum().reset_index()
bathrooms

/var/folders/gv/tfv3kbl11y58yd1f4j17_fqr0000gn/T/ipykernel_4721/928144867.py:9: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  bathrooms = bathrooms.str.split(' ', 1, expand=True)


,Nombre,count
0,0.00,33
1,0.25,17
2,0.50,355
3,0.75,198
4,1.00,2807
5,1.50,1928
6,1.75,1
7,2.00,1260
8,2.50,157
9,3.00,79


In [77]:
# QUESTION N°7

listing['len_description'] = listing['description'].apply(lambda x: len(str(x)))


corr = listing['len_description'].corr(listing['number_of_reviews'])
# Correlation : 0.19023440848679404
corr

0.19023440848679404

In [78]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342904 entries, 0 to 342903
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   listing_id     342904 non-null  int64 
 1   id             342904 non-null  int64 
 2   date           342904 non-null  object
 3   reviewer_id    342904 non-null  int64 
 4   reviewer_name  342904 non-null  object
 5   comments       342890 non-null  object
dtypes: int64(3), object(3)
memory usage: 15.7+ MB


In [79]:
# QUESTION N°8

listing_brut = pd.read_csv('./data/Netherlands/Amsterdam/listings-brut.csv')

fake_reviews = listing_brut.merge(reviews, left_on='id', right_on='listing_id')
fake_reviews = fake_reviews[["host_name","host_id", "reviewer_name", "reviewer_id"]].loc[fake_reviews["host_name"] == fake_reviews["reviewer_name"]].drop_duplicates()

In [80]:
annonces_fake = (len(fake_reviews) / len(reviews)) * 100
annonces_fake

0.04986818468142687

In [81]:
import pickle

reponses = {
    "1" : question1,
    "2" : question2,
    "3" : question3,
    "4" : g_room_type,
    "5" : prix,
    "6" : bathrooms,
    "7" : corr,
    "8" : annonces_fake
}


pickle_out = open("modelRid.pkl", "wb")
pickle.dump(reponses, pickle_out)
pickle_out.close()